In [1]:
!git clone https://github.com/chengtan9907/OpenSTL

Cloning into 'OpenSTL'...
remote: Enumerating objects: 2563, done.
remote: Counting objects: 100% (1075/1075), done.
remote: Compressing objects: 100% (284/284), done.
remote: Total 2563 (delta 904), reused 804 (delta 790), pack-reused 1488
Receiving objects: 100% (2563/2563), 3.01 MiB | 19.39 MiB/s, done.
Resolving deltas: 100% (1828/1828), done.


In [2]:
!pip install -r /content/OpenSTL/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 936.0/936.0 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.3/400.3 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 40.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.3/802.3 kB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.3/92.3 kB 4.3 MB/s et

In [3]:
!gdown --id 1v5ZDmnzVYP7Pb5D2Xz9j1ReqJ5j975WR

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1v5ZDmnzVYP7Pb5D2Xz9j1ReqJ5j975WR
From (redirected): https://drive.google.com/uc?id=1v5ZDmnzVYP7Pb5D2Xz9j1ReqJ5j975WR&confirm=t&uuid=d32fe547-4462-4189-810b-a7943dbb3efa
To: /content/t2m_2017_to_2018.nc
100% 144M/144M [00:00<00:00, 148MB/s]


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import xarray as xr
# from scipy.io import loadmat
# from fastai.vision.all import *
import gzip
import random
import glob
# import subprocess
from PIL import Image
from matplotlib.pyplot import imshow, figure
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

import torch
from torch import nn, optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
import torch.utils.data as Data
from torch.utils.data import Dataset
from torchvision import transforms, datasets
from datetime import datetime
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import math

In [2]:
os.chdir('/content/OpenSTL')

In [3]:
current_directory = os.getcwd()
print(f"Current Working Directory (Python): {current_directory}")

Current Working Directory (Python): /content/OpenSTL


In [4]:
ds = xr.open_mfdataset('/content/t2m_2017_to_2018.nc')
ds

<xarray.Dataset>
Dimensions:  (time: 17520, lat: 32, lon: 64)
Coordinates:
  * time     (time) datetime64[ns] 2017-01-01 ... 2018-12-31T23:00:00
  * lon      (lon) float64 0.0 5.625 11.25 16.88 ... 337.5 343.1 348.8 354.4
  * lat      (lat) float64 -87.19 -81.56 -75.94 -70.31 ... 75.94 81.56 87.19
Data variables:
    t2m      (time, lat, lon) float32 dask.array<chunksize=(17520, 32, 64), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2019-11-07 07:51:20 GMT by grib_to_netcdf-2.14.0: /opt/ecmw...

Perepare dataset

In [5]:
def slice_data(data, window_size):
  Xtemp = []
  for i in range(0, (len(data) - window_size)):
    Xtemp.append(data[i:i+window_size])
  return  np.array(Xtemp)

# def slice_data2(data, window_size):
#     data = np.array(data)
#     nrows = len(data) - window_size + 1
#     Xtemp = np.lib.stride_tricks.as_strided(data, shape=(nrows, window_size), strides=(data.strides[0], data.strides[0]))
#     return Xtemp

def data_splitmimo(x, y, split_ratio1=0.85, split_ratio2=0.93):
  '''
  convert to torch format and split train&test
  '''
  train_size = int(len(y) * split_ratio1)
  val_size = int(len(y) * split_ratio2)

  x_data = Variable(torch.Tensor(np.array(x)))
  y_data = Variable(torch.Tensor(np.array(y)))

  x_train = Variable(torch.Tensor(np.array(x[0:train_size])))
  y_train = Variable(torch.Tensor(np.array(y[0:train_size])))

  x_val = Variable(torch.Tensor(np.array(x[train_size:val_size])))
  y_val = Variable(torch.Tensor(np.array(y[train_size:val_size])))

  x_test = Variable(torch.Tensor(np.array(x[val_size:len(x)])))
  y_test = Variable(torch.Tensor(np.array(y[val_size:len(y)])))
  return x_data, y_data, x_train, y_train, x_test, y_test, x_val, y_val

def shift_data(data, n_frames):
    print(data.shape)
    data = np.expand_dims(data, axis=2)
    X = data[:, 0:n_frames,:, :,  :]
    y = data[:, n_frames:, :,:,  :]
    return X, y

In [6]:
X = ds.t2m.data
X = X[:3000,:,:] # Change to run it for full exp
print(X.shape)
# normalization
data = (X - X.mean()) / X.std()
#data = (X - X.min()) / (X.max() - X.min())


(3000, 32, 64)


In [8]:
data = slice_data(data, 24)
X, Y = shift_data(data, 12)

(2976, 24, 32, 64)


In [9]:
x_data, y_data, x_train, y_train, x_test, y_test, x_val, y_val = data_splitmimo(X, Y)

# Inspect the dataset.
print("Train Dataset Shapes: " + str(x_train.shape) + ", " + str(y_train.shape))
print("Val Dataset Shapes: " + str(x_val.shape) + ", " + str(y_val.shape))
print("Test Dataset Shapes: " + str(x_test.shape) + ", " + str(y_test.shape))

Train Dataset Shapes: torch.Size([2529, 12, 1, 32, 64]), torch.Size([2529, 12, 1, 32, 64])
Val Dataset Shapes: torch.Size([238, 12, 1, 32, 64]), torch.Size([238, 12, 1, 32, 64])
Test Dataset Shapes: torch.Size([209, 12, 1, 32, 64]), torch.Size([209, 12, 1, 32, 64])


In [10]:
class CustomDataset(Dataset):
    def __init__(self, X, Y, normalize=False, data_name='custom'):
        super(CustomDataset, self).__init__()
        self.X = X
        self.Y = Y
        self.mean = None
        self.std = None
        self.data_name = data_name

        if normalize:
            # get the mean/std values along the channel dimension
            mean = data.mean(axis=(0, 1, 2, 3)).reshape(1, 1, -1, 1, 1)
            std = data.std(axis=(0, 1, 2, 3)).reshape(1, 1, -1, 1, 1)
            data = (data - mean) / std
            self.mean = mean
            self.std = std

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, index):
        data = torch.tensor(self.X[index]).float()
        labels = torch.tensor(self.Y[index]).float()
        return data, labels

In [11]:
train_dataset = CustomDataset(x_train, y_train)
val_dataset = CustomDataset(x_val, y_val)
test_dataset = CustomDataset(x_test, y_test)

In [12]:
batch_size = 8
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                        batch_size=batch_size,
                                        shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                        batch_size=batch_size,
                                        shuffle=False)
val_loader = torch.utils.data.DataLoader(dataset=val_dataset,
                                        batch_size=batch_size,
                                        shuffle=False)

In [13]:
# Models
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    torch.cuda.set_device(0)
    print(torch.cuda.get_device_name())

Tesla T4


## 3. Train and evaluate different models

In [14]:
from openstl.api import BaseExperiment
from openstl.utils import create_parser, default_parser

## ConvLSTM

In [ ]:
custom_training_config_ConvLSTM = {
    'pre_seq_length': 12,
    'aft_seq_length': 12,
    'total_length': 24,
    'batch_size': batch_size,
    'val_batch_size': batch_size,
    'epoch': 10,
    'lr': 0.001,
    'metrics': ['mae', 'mse'],

    'ex_name': 'ConvLSTM',
    'dataname': 'custom',
    'in_shape': [12, 1, 32, 64],
}


custom_model_config_ConvLSTM = {
    'method': 'ConvLSTM',
    'reverse_scheduled_sampling': 0,
    'r_sampling_step_1': 25000,
    'r_sampling_step_2': 50000,
    'r_exp_alpha': 5000,
    'scheduled_sampling': 1,
    'sampling_stop_iter': 50000,
    'sampling_start_value': 1.0,
    'sampling_changing_rate': 0.00002,
    'num_hidden': '128,128,128',
    'filter_size': 5,
    'stride': 1,
    'patch_size': 2,
    'layer_norm': 0,
    'lr': 5e-4,
    'batch_size': batch_size,
    # 'sched': 'cosine',
    'warmup_epoch': 0,
    'config_file': '/content/OpenSTL/configs/weather/t2m_5_625/ConvLSTM.py'
}

In [ ]:
args = create_parser().parse_args([])
config = args.__dict__

# update the training config
config.update(custom_training_config_ConvLSTM)
# update the model config
config.update(custom_model_config_ConvLSTM)
# fulfill with default values
default_values = default_parser()
for attribute in default_values.keys():
    if config[attribute] is None:
        config[attribute] = default_values[attribute]

expConvLSTM = BaseExperiment(args, dataloaders=(train_loader, val_loader, test_loader), strategy='auto')

### 3.3 Start training and evaluation

With everything in place, we are now ready to start training and evaluation.

Here we go!

In [ ]:
print('>'*35 + ' training ' + '<'*35)
expConvLSTM.train()

print('>'*35 + ' testing  ' + '<'*35)
expConvLSTM.test()

## PhyDNET

In [ ]:
custom_training_config_PhyDNet = {
    'pre_seq_length': 12,
    'aft_seq_length': 12,
    'total_length': 24,
    'batch_size': batch_size,
    'val_batch_size': batch_size,
    'epoch': 20,
    'lr': 0.001,
    'metrics': ['mae', 'mse'],

    'ex_name': 'PhyDNet',
    'dataname': 'custom',
    'in_shape': [12, 1, 32, 64],
}


custom_model_config_PhyDNet = {
    'method': 'PhyDNet',
    'patch_size': 2,
    'lr': 1e-4,
    'batch_size': batch_size,
    # 'sched': 'cosine',
    'warmup_epoch': 0,
    'config_file': '/content/OpenSTL/configs/weather/t2m_5_625/PhyDNet.py'
}

In [ ]:
args = create_parser().parse_args([])
config = args.__dict__

# update the training config
config.update(custom_training_config_PhyDNet)
# update the model config
config.update(custom_model_config_PhyDNet)
# fulfill with default values
default_values = default_parser()
for attribute in default_values.keys():
    if config[attribute] is None:
        config[attribute] = default_values[attribute]

expPhyDNet = BaseExperiment(args, dataloaders=(train_loader, val_loader, test_loader), strategy='auto')

### 3.3 Start training and evaluation

With everything in place, we are now ready to start training and evaluation.

Here we go!

In [ ]:
print('>'*35 + ' training ' + '<'*35)
expPhyDNet.train()

print('>'*35 + ' testing  ' + '<'*35)
expPhyDNet.test()

## PredRNN

In [ ]:
custom_training_config_PredRNN = {
    'pre_seq_length': 12,
    'aft_seq_length': 12,
    'total_length': 24,
    'batch_size': batch_size,
    'val_batch_size': batch_size,
    'epoch': 10,
    'lr': 0.001,
    'metrics': ['mae', 'mse'],

    'ex_name': 'PredRNN',
    'dataname': 'custom',
    'in_shape': [12, 1, 32, 64],
}


custom_model_config_PredRNN = {
    'method': 'PredRNN',
    'reverse_scheduled_sampling': 0,
    'r_sampling_step_1': 25000,
    'r_sampling_step_2': 50000,
    'r_exp_alpha': 5000,
    'scheduled_sampling': 1,
    'sampling_stop_iter': 50000,
    'sampling_start_value': 1.0,
    'sampling_changing_rate': 0.00002,
    'num_hidden': '128,128,128',
    'filter_size': 5,
    'stride': 1,
    'patch_size': 2,
    'layer_norm': 0,
    'lr': 5e-4,
    'batch_size': batch_size,
    # 'sched': 'cosine',
    'warmup_epoch': 0,
    'config_file': '/content/OpenSTL/configs/weather/t2m_5_625/PredRNN.py'
}

In [ ]:
args = create_parser().parse_args([])
config = args.__dict__

# update the training config
config.update(custom_training_config_PredRNN)
# update the model config
config.update(custom_model_config_PredRNN)
# fulfill with default values
default_values = default_parser()
for attribute in default_values.keys():
    if config[attribute] is None:
        config[attribute] = default_values[attribute]

expRNN = BaseExperiment(args, dataloaders=(train_loader, val_loader, test_loader), strategy='auto')

### 3.3 Start training and evaluation

With everything in place, we are now ready to start training and evaluation.

Here we go!

In [ ]:
print('>'*35 + ' training ' + '<'*35)
expRNN.train()

print('>'*35 + ' testing  ' + '<'*35)
expRNN.test()

## PredRNN ++

In [ ]:
custom_training_config_PredRNNpp = {
    'pre_seq_length': 12,
    'aft_seq_length': 12,
    'total_length': 24,
    'batch_size': batch_size,
    'val_batch_size': batch_size,
    'epoch': 10,
    'lr': 0.001,
    'metrics': ['mae', 'mse'],

    'ex_name': 'PredRNNpp',
    'dataname': 'custom',
    'in_shape': [12, 1, 32, 64],
}


custom_model_config_PredRNNpp = {
     'method': 'PredRNNpp',
    'reverse_scheduled_sampling': 0,
    'r_sampling_step_1': 25000,
    'r_sampling_step_2': 50000,
    'r_exp_alpha': 5000,
    'scheduled_sampling': 1,
    'sampling_stop_iter': 50000,
    'sampling_start_value': 1.0,
    'sampling_changing_rate': 0.00002,
    'num_hidden': '128,128,128',
    'filter_size': 5,
    'stride': 1,
    'patch_size': 2,
    'layer_norm': 0,
    'lr': 5e-4,
    'batch_size': batch_size,
    # 'sched': 'cosine',
    'warmup_epoch': 0,
    'config_file': '/content/OpenSTL/configs/weather/t2m_5_625/PredRNNpp.py'
}

In [ ]:
args = create_parser().parse_args([])
config = args.__dict__

# update the training config
config.update(custom_training_config_PredRNNpp)
# update the model config
config.update(custom_model_config_PredRNNpp)
# fulfill with default values
default_values = default_parser()
for attribute in default_values.keys():
    if config[attribute] is None:
        config[attribute] = default_values[attribute]

expRNNpp = BaseExperiment(args, dataloaders=(train_loader, val_loader, test_loader), strategy='auto')

### 3.3 Start training and evaluation

With everything in place, we are now ready to start training and evaluation.

Here we go!

In [ ]:
print('>'*35 + ' training ' + '<'*35)
expRNNpp.train()

print('>'*35 + ' testing  ' + '<'*35)
expRNNpp.test()

## TAU

In [15]:
custom_training_config_TAU = {
    'pre_seq_length': 12,
    'aft_seq_length': 12,
    'total_length': 24,
    'batch_size': batch_size,
    'val_batch_size': batch_size,
    'epoch': 10,
    'lr': 0.001,
    'metrics': ['mae', 'mse'],

    'ex_name': 'TAU',
    'dataname': 'custom',
    'in_shape': [12, 1, 32, 64],
}


custom_model_config_TAU = {
     'method': 'TAU',
    'spatio_kernel_enc': 3,
    'spatio_kernel_dec': 3,
    'model_type': 'tau',
    'hid_S': 32,
    'hid_T': 128,
    'N_T': 8,
    'N_S': 2,
    'alpha': 0.1,
    'lr': 5e-3,
    'batch_size': batch_size,
    'drop_path': 0.1,
    'warmup_epoch': 0,
    'config_file': '/content/OpenSTL/configs/weather/t2m_5_625/TAU.py'
}

In [16]:
args = create_parser().parse_args([])
config = args.__dict__

# update the training config
config.update(custom_training_config_TAU)
# update the model config
config.update(custom_model_config_TAU)
# fulfill with default values
default_values = default_parser()
for attribute in default_values.keys():
    if config[attribute] is None:
        config[attribute] = default_values[attribute]

expTAU = BaseExperiment(args, dataloaders=(train_loader, val_loader, test_loader), strategy='auto')

INFO:lightning_fabric.utilities.seed:Seed set to 42
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


### 3.3 Start training and evaluation

With everything in place, we are now ready to start training and evaluation.

Here we go!

In [17]:
print('>'*35 + ' training ' + '<'*35)
expTAU.train()

print('>'*35 + ' testing  ' + '<'*35)
expTAU.test()

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> training <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<


INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Environment info:
------------------------------------------------------------
sys.platform: linux
Python: 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]
CUDA available: True
CUDA_HOME: /usr/local/cuda
NVCC: Build cuda_12.2.r12.2/compiler.33191640_0
GPU 0: Tesla T4
GCC: gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
PyTorch: 2.3.0+cu121
PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v3.3.6 (Git Hash 86e6af5974177e513fd3fee58425e1063e7f1361)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 12.1
  - NVCC architecture flags: -gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_70,code=sm_70;-gencode;arch=compute_75,code=sm_75;-gencode;arch=compute_80,code=sm_80;-gencode;

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

...

Epoch 8: Lr: 0.0002461 | Train Loss: 0.0013565 | Vali Loss: 0.0054391


Validation: |          | 0/? [00:00<?, ?it/s]

Epoch 9: Lr: 0.0000000 | Train Loss: 0.0011839 | Vali Loss: 0.0054452
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> testing  <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<


Testing: |          | 0/? [00:00<?, ?it/s]

mse:11.80522346496582, mae:101.16151428222656


## SimVP

In [18]:
custom_training_config_SimVP = {
    'pre_seq_length': 12,
    'aft_seq_length': 12,
    'total_length': 24,
    'batch_size': batch_size,
    'val_batch_size': batch_size,
    'epoch': 20,
    'lr': 0.001,
    'metrics': ['mse', 'mae'],

    'ex_name': 'custom_exp',
    'dataname': 'custom',
    'in_shape': [12, 1, 32, 64],
}

custom_model_config_SimVP = {
    'method': 'SimVP',
    'model_type': 'gSTA',
    'N_S': 4,
    'N_T': 8,
    'hid_S': 64,
    'hid_T': 128
}

In [19]:
args = create_parser().parse_args([])
config = args.__dict__

# update the training config
config.update(custom_training_config_SimVP)
# update the model config
config.update(custom_model_config_SimVP)
# fulfill with default values
default_values = default_parser()
for attribute in default_values.keys():
    if config[attribute] is None:
        config[attribute] = default_values[attribute]

expSimVP = BaseExperiment(args, dataloaders=(train_loader, val_loader, test_loader), strategy='auto')

### 3.3 Start training and evaluation

With everything in place, we are now ready to start training and evaluation.

Here we go!

In [20]:
print('>'*35 + ' training ' + '<'*35)
expSimVP.train()

print('>'*35 + ' testing  ' + '<'*35)
expSimVP.test()

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> training <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
Environment info:
------------------------------------------------------------
sys.platform: linux
Python: 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]
CUDA available: True
CUDA_HOME: /usr/local/cuda
NVCC: Build cuda_12.2.r12.2/compiler.33191640_0
GPU 0: Tesla T4
GCC: gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
PyTorch: 2.3.0+cu121
PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v3.3.6 (Git Hash 86e6af5974177e513fd3fee58425e1063e7f1361)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 12.1
  - NVCC architecture flags: -gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_70,code=sm_70;

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Epoch 18: Lr: 0.0000125 | Train Loss: 0.0006899 | Vali Loss: 0.0092456


Validation: |          | 0/? [00:00<?, ?it/s]

Epoch 19: Lr: 0.0000000 | Train Loss: 0.0006640 | Vali Loss: 0.0092667
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> testing  <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<


Testing: |          | 0/? [00:00<?, ?it/s]

mse:21.217742919921875, mae:142.18316650390625
